# Facial Expression Analysis: Action Unit Intensity Estimation

- The objective is to estimate facial emotion from expression using facial *Action Units (AUs)* intensity and *FACS (Facial Action Coding System)* in real-time.
- To accomplish this, used a large-scale pre-trained network (**Masked Auto-encoder**) and performed feature-wise **knowledge distillation** with task-specific fine-tuning on a lightweight model (**ResNet-18**) to get facial *Action Unit* intensity in real-time.
- Designed visual emotion stimuli to induce different emotions and simultaneously acquire eye coordinates and face video to estimate eye-motion features and facial micro-expression corresponding to the shown emotion stimulus.

## Method

- A **Masked Auto-encoder** (*MAE*) is used which was pre-trained in self-supervised manner (masked input image reconstruction task) with EmotioNet dataset. Subsequently, only the encoder part is extracted and attached to a linear classification layer, and further pre-trained on AffectNet and FFHQ dataset which are large facial expression datasets before finally fine-tuning on the DISFA dataset for facial *Action Unit* intensity estimation.
- Now, since the *MAE* is a large model, to perform faster and real-time estimation, employed **feature-wise knowledge distillation** to transfer the *teacher model*'s (**MAE**) knowledge to a lightweight *student model* (**ResNet-18**).
- The *ResNet-18* model with linear classification layer attached, is first pre-trained on the same AffectNet and FFHQ datasets and then fine-tuned with simultaneous knowledge distillation from teacher model on DISFA dataset for facial *Action-Unit* intensity estimation.
- Using the facial *Action Unit* intensity values, their activation is assessed and based on **FACS (Facial Action Coding System)**  which defines a relation between the action units and emotion, the overall facial emotion is estimated.

<div align="center">
  <img src="Method%20Image.jpg" width="1000" />
  <img src="images/Facial_Action_Unit_Intensity_Model_Architecture.png" width="1000" />
</div>

### Losses:

1. **Feature Matching Loss**: A *MSE* loss between hidden feature layers of the *teacher* model and the *student* model. \
$\mathcal{L}_{FM} = \left\|f_{T}  - \textbf{I}(f_{S})\right\| ^{2}$

2. **KL Divergence Loss**: between *teacher* model's output for (i) the input face image and  (ii) the *student* model's hidden feature layer input to the *teacher* linear classifiation layer. \
$\mathcal{L}_{KLD} = - \widehat{y}_{T}  ~ \text{log} (\frac{\widehat{y}_{T}}{\widehat{y}_{S}})$

3. **Task Loss**: The training *MSE* loss for the *student* network. \
$\mathcal{L}_{Task} = \left\|\widehat{y}  - y\right\| ^{2}$

**Overall Loss:** $\mathcal{L} = \mathcal{L}_{FM} + \alpha\mathcal{L}_{Task} + \beta\mathcal{L}_{KLD}$

- Performance on DISFA datast for facial *Action Unit* intensity estimation task: 

| Method                 | PCC        |  MAE     | MSE      |
| ---------------------- | -----------| -------- | -------- |
| ResNet-18              |   0.518    |  0.278   |  0.352   |
| ResNet-18 + Pre-Train  |   0.614    |  0.236   |  0.260   |
| ResNet-18 + FM Distill |   0.628    |  0.244   |  0.260   |
| MAE + Pre-Train        |   0.674    |  0.202   |  0.270   |
